In [ ]:
import re
import os

from termcolor import colored
import pandas as pd
import logging
import pyxatu
import plotly.graph_objects as go
import pandas_gbq
from datetime import timedelta

pd.set_option('display.max_colwidth', None)

xatu = pyxatu.PyXatu()

def set_google_credentials(CONFIG, GOOGLE_CREDENTIALS):
    try:
        os.environ['GOOGLE_APPLICATION_CREDENTIALS']
    except:
        print(f"setting google credentials as global variable...")
        os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = CONFIG \
        + GOOGLE_CREDENTIALS or input("No Google API credendials file provided." 
        + "Please specify path now:\n")
        
set_google_credentials("../../config/","google-creds.json")


In [ ]:
maxslot = int(xatu.execute_query("""SELECT distinct max(slot) FROM canonical_beacon_block_execution_transaction
where meta_network_name = 'mainnet' 
""")[0])
maxslot

In [ ]:
df = xatu.execute_query("""
SELECT distinct count(distinct hash) slots, blob_gas/131072 as blob_gas 
FROM (SELECT distinct hash, blob_gas, slot FROM canonical_beacon_block_execution_transaction
where meta_network_name = 'mainnet' and slot >= 10526178 and blob_gas > 0)
group by blob_gas
order by blob_gas
"""
, columns = "nr_transactions, blobs")
df

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=df['blobs'],
    y=df['nr_transactions'],
    name='Blobs Per Transaction',
    marker_color='firebrick'
))

fig.update_layout(
    title='Blobs Per Transaction <span style="font-size: 15;">({} - {})</span>'.format(xatu.helpers.slot_to_day(10526178), xatu.helpers.slot_to_day(maxslot)),
    xaxis_title=f'Blobs Per Transaction',
    yaxis_title='nr. of transactions',
    barmode='stack',
    font=dict(size=14, family='Ubuntu Mono', color='black'),
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin={"t": 100, "b": 80, "r": 50, "l": 50},
    xaxis=dict(
        showgrid=True,
        gridcolor='lightgrey',
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor='lightgrey',
    ),
    height=350,
    width=800
)

fig.show()

In [ ]:
df = xatu.execute_query("""
SELECT distinct count(distinct slot) slots, ifnull(blob_gas/131072, 0) as blob_gas FROM (
SELECT slot, sum(blob_gas) blob_gas FROM (
    SELECT distinct hash,  slot, blob_gas 
    FROM canonical_beacon_block_execution_transaction
    where meta_network_name = 'mainnet' 
    and slot >= 10526178 
    )
    group by slot
    )
    where blob_gas >= 0
    group by blob_gas
    order by blob_gas
"""
, columns = "nr_blocks, blobs")
df

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=df['blobs'],
    y=df['nr_blocks'],
    name='Blobs Per Block',
    marker_color='firebrick'
))

fig.update_layout(
    title='Blobs Per Block <span style="font-size: 15;">({} - {})</span>'.format(xatu.helpers.slot_to_day(10526178), xatu.helpers.slot_to_day(maxslot)),
    xaxis_title=f'Blobs Per Block',
    yaxis_title='nr. of blocks',
    barmode='stack',
    font=dict(size=14, family='Ubuntu Mono', color='black'),
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin={"t": 100, "b": 80, "r": 50, "l": 50},
    xaxis=dict(
        showgrid=True,
        gridcolor='lightgrey',
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor='lightgrey',
    ),
    height=350,
    width=800
)

fig.show()

In [ ]:
blobs = xatu.execute_query("""
SELECT DISTINCT slot,countDistinct(blob_index) as blobs
FROM beacon_api_eth_v1_events_blob_sidecar FINAL WHERE slot_start_date_time > NOW() - INTERVAL '396 days' 
AND meta_network_name = 'mainnet' GROUP BY slot ORDER BY slot
""", columns="slot, blobs")
blobs["slot"] = blobs["slot"].astype(int)
all_slots = pd.DataFrame({'slot': range(blobs.slot.min(), blobs.slot.max() + 1)})
df = pd.merge(all_slots, blobs, on='slot', how='left').fillna({'blobs': 0})
df = df.reset_index(drop=True)
df.sort_values("slot", inplace=True)
df

In [ ]:
excess = xatu.execute_query(f"""
SELECT slot, execution_payload_excess_blob_gas FROM canonical_beacon_block where meta_network_name = 'mainnet'
and slot >= {blobs.slot.min()} and slot <= {blobs.slot.max()} 
order by slot
""", columns="slot, excess_blob_gas")
excess.head(20)

In [ ]:
df = pd.merge(df, excess, how="left", left_on="slot", right_on="slot").fillna(0)

In [ ]:
import math
BLOB_BASE_FEE_UPDATE_FRACTION = 3338477
df["basefee"] = df["excess_blob_gas"].apply(lambda x: math.floor(math.exp(x/BLOB_BASE_FEE_UPDATE_FRACTION)))

In [ ]:
mev = pd.read_parquet("mev_data_.parquet")

In [ ]:
query = """
   SELECT distinct slot, max(value) as value FROM `ethereum-data-nero.eth.mevboost_db` 
WHERE TIMESTAMP_TRUNC(date, DAY) >= TIMESTAMP("2024-01-01") 
group by slot
order by slot
"""
mev = pandas_gbq.read_gbq(query)
mev.to_parquet("mev_data_.parquet", index=None)

In [ ]:
mev.dropna(inplace=True)
mev["value_gwei"] = mev["value"].apply(lambda x:  int(x) / 1e9)
mev

In [ ]:
df = pd.merge(df, mev[["slot", "value_gwei"]], left_on="slot", right_on="slot")

In [ ]:
query = f"""
SELECT distinct tx_hash, nr_blobs, gas_used, gas_price FROM `ethereum-data-nero.ethdata.ethereum_transactions` 
where type = 3 and block_number >= 19426589
"""
gu = pandas_gbq.read_gbq(query)
gu.to_parquet("gu_data.parquet", index=None)

In [ ]:
gu = pd.read_parquet("gu_data.parquet")

In [ ]:
query = f"""
SELECT block_number, sum(gas_used) block_gas_used ,sum(gas_used*(gas_price/1000000000)) block_cost 
FROM (
SELECT distinct block_number, tx_hash, gas_used, gas_price FROM `ethereum-data-nero.ethdata.ethereum_transactions` 
where type = 2 and block_number >= 19426589
)
group by block_number
"""
gu2 = pandas_gbq.read_gbq(query)
gu2.to_parquet("gu2_data.parquet", index=None)

In [ ]:
gu2 = pd.read_parquet("gu2_data.parquet")

In [ ]:
blobgas = xatu.execute_query("""SELECT distinct slot, hash, blob_gas FROM canonical_beacon_block_execution_transaction
where blob_gas > 0 and meta_network_name = 'mainnet' and slot >= 10526178
"""
, columns = "slot, tx_hash, blob_gas")
blobgas

In [ ]:
blobgas["blobs"] = blobgas["blob_gas"] // 131072
blobgas.groupby("tx_hash")["blobs"].sum().reset_index().groupby("blobs")["tx_hash"].count()

In [ ]:
base_fee = xatu.execute_query(
    f"""
        SELECT slot, execution_payload_block_number, execution_payload_base_fee_per_gas 
        FROM canonical_beacon_block where meta_network_name = 'mainnet'
        and slot >= {int(blobgas.slot.min())}
    """
, columns="slot, block_number, base_fee")
base_fee

In [ ]:
blobgas = pd.merge(blobgas, gu, left_on="tx_hash", right_on="tx_hash")
blobgas["blob_tx_cost"] = (blobgas["gas_used"] * blobgas["gas_price"] / 1e9).astype(int)
blobgas_base = pd.merge(blobgas, base_fee, left_on="slot", right_on="slot")
blobgas_base["basefee_used"] = blobgas_base["gas_used"] * blobgas_base["base_fee"] / 1e9
blobgas_base["prio"] = blobgas_base["blob_tx_cost"] - blobgas_base["basefee_used"]

In [ ]:
blobgas_gr = blobgas_base.groupby("slot")[["prio"]].sum().reset_index()
blobgas_gr = pd.merge(mev[["slot", "value_gwei"]], blobgas_gr, left_on="slot", right_on="slot")
blobgas_gr

In [ ]:
blobgas_gr.prio.median()/1e9

In [ ]:
blobgas_gr.prio.mean()/1e9

In [ ]:
blobgas_gr["date"] = blobgas_gr.slot.apply(lambda x: xatu.helpers.slot_to_day(x))
df2 = blobgas_gr.groupby("date")[["value_gwei", "prio"]].mean().reset_index()
df2

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df2["date"],
        y=df2["prio"] / 1e9,
        mode="lines",
        name="Prio Fees from Blobs",
        line=dict(color="firebrick", width=2),
        fill="tonexty",
        fillpattern=dict(
            shape="/",        
            bgcolor="white", 
            fgcolor="firebrick"
        )
    )
)
fig.add_trace(
    go.Scatter(
        x=df2["date"],
        y=df2["value_gwei"] / 1e9,
        mode="lines",
        name="MEV-Boost Payment",
        line=dict(color="black", width=2),
        fill="tonexty",
        fillpattern=dict(
            shape=".",
            bgcolor="white",
            fgcolor="black"
        )
    )
)

fig.update_layout(
    title={
        'text': "Avg. Profits Per Block: MEV-Boost vs. Blobs",
    },
    xaxis_title=None,
    yaxis_title="ETH (log scale)",
    font=dict(
        family="Ubuntu mono",
        size=12,
        color="black"
    ),
    plot_bgcolor="white",
    paper_bgcolor="white",
    xaxis=dict(
        showgrid=True,
        gridcolor="lightgray",
        zeroline=False,
        tickangle=45
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor="lightgray",
        zeroline=False,
        type="log"
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1,
        bgcolor="rgba(255, 255, 255, 0.5)",
        bordercolor="lightgray",
        borderwidth=1
    ),
    margin=dict(l=60, r=40, t=80, b=80)
)

fig.show()

In [ ]:
blobgas.columns = ["slot", "tx_hash", "blob_gas", "blobs", "nr_blobs", "tx_gas_used", "gas_price", "tx_cost_gwei"]

In [ ]:
blobgas_base = pd.merge(blobgas, base_fee, left_on="slot", right_on="slot")
blobgas_base

In [ ]:
blobgas_gr = pd.merge(blobgas, base_fee, left_on="slot", right_on="slot")
blobgas_gr

In [ ]:
blobgas_gr["prio_fee"] = (blobgas_gr["tx_cost_gwei"] - (blobgas_gr["tx_gas_used"] * blobgas_gr["base_fee"]) / 1e9).astype(int)
blobgas_gr["basefee_total"] = (blobgas_gr["base_fee"] * blobgas_gr["tx_gas_used"] // 1e9).astype(int)
blobgas_gr

In [ ]:
blobgas_base = blobgas_gr[["blobs", "prio_fee"]]
blobgas_base["prio_fee"] =  blobgas_base["prio_fee"]/1e9

In [ ]:
blobgas_base.prio_fee.mean(),blobgas_base.prio_fee.median()

In [ ]:
import plotly.graph_objects as go
import numpy as np

unique_blobs = sorted(blobgas_base['blobs'].unique())
x_positions = list(range(len(unique_blobs)))

fig = go.Figure()

box_half_width = 0.3

for i, b in enumerate(unique_blobs):
    blob_data = blobgas_base[blobgas_base['blobs'] == b]
    avg_val = blob_data['prio_fee'].mean()
    
    fig.add_trace(
        go.Box(
            x=[i] * len(blob_data),
            y=blob_data['prio_fee'],
            name=None,
            showlegend=False,
            boxpoints=False,
            marker_color='firebrick',
            line_color='firebrick'
        )
    )
    
    fig.add_shape(
        type="line",   
        x0=i - box_half_width,
        x1=i + box_half_width,
        y0=avg_val,
        y1=avg_val,
        line=dict(color="black", width=2, dash="dash"),
        xref="x",
        yref="y"
    )

fig.add_trace(
    go.Scatter(
        x=[None],
        y=[None],
        mode='lines',
        line=dict(color="black", width=2, dash="dash"),
        name="avg. prio fee (gwei)"
    )
)

fig.update_layout(
    title={'text': "Priority Fee of Blob Transactions"},
    xaxis=dict(
        title="Blobs/Transaction",
        tickmode='array',
        tickvals=x_positions,
        ticktext=unique_blobs,
        showgrid=True,
        gridcolor="lightgray",
        zeroline=False,
    ),
    yaxis=dict(
        title="Prio Fee (log scale)",
        showgrid=True,
        gridcolor="lightgray",
        zeroline=False,
        type="log",
        ticksuffix=" ETH",
        tickformat=".3",
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    font=dict(
        family="Ubuntu mono",
        size=14,
        color="black"
    ),
    plot_bgcolor="white",
    paper_bgcolor="white",
    margin=dict(l=60, r=40, t=80, b=80)
)

fig.show()

In [ ]:
blobgas_gr["prio_fee_per_gas"] = blobgas_gr["prio_fee"] / blobgas_gr["tx_gas_used"] 
blobgas_gr["basefee_total_per_gas"] = blobgas_gr["basefee_total"] / blobgas_gr["tx_gas_used"] 

In [ ]:
blobgas_gra = blobgas_gr.groupby("slot")[["prio_fee", "basefee_total"]].sum().reset_index()
blobgas_gra["date"] = blobgas_gra.slot.apply(lambda x: xatu.helpers.slot_to_day(x))
df2 = blobgas_gra.groupby("date")[["prio_fee", "basefee_total"]].mean().reset_index()
df3 = blobgas_gra.groupby("date")[["prio_fee", "basefee_total"]].median().reset_index()
df2["prio_fee_per"] = df2["prio_fee"] / (df2["prio_fee"] + df2["basefee_total"])
df3["prio_fee_per"] = df3["prio_fee"] / (df3["prio_fee"] + df3["basefee_total"])
df2["basefee_total_per"] = df2["basefee_total"] / (df2["prio_fee"] + df2["basefee_total"])
df3["basefee_total_per"] = df3["basefee_total"] / (df3["prio_fee"] + df3["basefee_total"])
df2

In [ ]:
blobgas_gra.prio_fee.mean()/1e9

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df2["date"],
        y=df2["prio_fee"]/1e9,
        mode="lines",
        name="Priority Fee",
        line=dict(color="firebrick", width=2),
        fill="tonexty",
        fillpattern=dict(
            shape="/",
            bgcolor="white",
            fgcolor="firebrick"
        )
    )
)
fig.add_trace(
    go.Scatter(
        x=df2["date"],
        y=df2["prio_fee"]/1e9+ df2["basefee_total"]/1e9,
        mode="lines",
        name="Total (Basefee + Priority Fee)",
        line=dict(color="black", width=2),
        fill="tonexty",
        fillpattern=dict(
            shape=".",
            bgcolor="white",
            fgcolor="black"
        )
    )
)
#fig.add_trace(
#    go.Scatter(
#        x=gu2["date"],
#        y=gu2["block_prio_fee"]/1e9,
#        mode="lines",
#        name="Type 2",
#        line=dict(color="royalblue", width=2),
#        fill="tonexty",  # Fills the area from the trace to y=0
#        fillpattern=dict(
#            shape=".",         # Use a '//' hatch pattern
#            bgcolor="white", # Base color (will be blended with the pattern)
#            fgcolor="black"      # Color of the hatch lines
#        )
#    )
#)

# Update layout for a clean, professional look
fig.update_layout(
    
    title='Prio Fee vs Base Fee for Type 3 Transactions <span style="font-size: 15;">({} - {})</span>'.format(xatu.helpers.slot_to_day(blobgas_gr.slot.min()), xatu.helpers.slot_to_day(blobgas_gr.slot.max())),

    xaxis_title=None,
    yaxis_title="ETH",
    font=dict(
        family="Ubuntu mono",
        size=12,
        color="black"
    ),
    plot_bgcolor="white",
    paper_bgcolor="white",
    xaxis=dict(
        showgrid=True,
        gridcolor="lightgray",
        zeroline=False,
        tickangle=45
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor="lightgray",
        zeroline=False
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=0.98,
        xanchor="left",
        x=0.01,
        bgcolor="rgba(255, 255, 255, 0.5)",
        bordercolor="lightgray",
        borderwidth=1
    ),
    margin=dict(l=60, r=40, t=80, b=80)
)

fig.show()


In [ ]:
blobgas_gra = blobgas_gr.groupby("slot")[["prio_fee_per_gas", "basefee_total_per_gas"]].sum().reset_index()
blobgas_gra["date"] = blobgas_gra.slot.apply(lambda x: xatu.helpers.slot_to_day(x))
df2 = blobgas_gra.groupby("date")[["prio_fee_per_gas", "basefee_total_per_gas"]].mean().reset_index()
df3 = blobgas_gra.groupby("date")[["prio_fee_per_gas", "basefee_total_per_gas"]].median().reset_index()
df2["prio_fee_per_gas_per"] = df2["prio_fee_per_gas"] / (df2["prio_fee_per_gas"] + df2["basefee_total_per_gas"])
df3["prio_fee_per_gas_per"] = df3["prio_fee_per_gas"] / (df3["prio_fee_per_gas"] + df3["basefee_total_per_gas"])
df2["basefee_total_per_gas_per"] = df2["basefee_total_per_gas"] / (df2["prio_fee_per_gas"] + df2["basefee_total_per_gas"])
df3["basefee_total_per_gas_per"] = df3["basefee_total_per_gas"] / (df3["prio_fee_per_gas"] + df3["basefee_total_per_gas"])
df2

In [ ]:
gu2 = pd.read_parquet("gu2_data.parquet")
gu2 = pd.merge(base_fee, gu2, left_on="block_number", right_on="block_number")
gu2["block_basefee"] = gu2["block_gas_used"] * gu2["base_fee"] / 1e9
gu2["block_prio_fee"] = (gu2["block_cost"] - gu2["block_basefee"]) #/ gu2["block_gas_used"]
#gu2["block_basefee"] = (gu2["block_basefee"]) / gu2["block_gas_used"]
gu2

medianprio = gu2.block_prio_fee.median() / 1e9
float(medianprio)

In [ ]:
all_slots = pd.DataFrame({'slot': range(blobgas_gr.slot.min(), blobgas_gr.slot.max() + 1)})
df = pd.merge(all_slots, blobgas_gr, on='slot', how='left').fillna(0)
df = df.reset_index(drop=True)
df.sort_values("slot", inplace=True)
df

In [ ]:
df.prio_fee.sum()

In [ ]:
df = df.groupby("slot")[["blobs", "prio_fee"]].sum().reset_index()
df

In [ ]:
df.prio_fee.sum()/df.blobs.sum()/1e9

In [ ]:
df.prio_fee.median()/1e9 * 0.7

In [ ]:
import pandas as pd

# Assuming blobgas_gr is already defined
l = len(df)
ll = len(range(df.slot.min(), df.slot.max()+1))#blobgas_gr.slot.nunique()

results = []

total_prio_fee = df.prio_fee.sum()
for i in range(0, 7):
    # Sum of prio_fee for all rows
    # Sum of prio_fee for rows where 'blobs' is less than or equal to i
    filtered_prio_fee = df[df["blobs"] <= i].prio_fee.sum()
    print(total_prio_fee-filtered_prio_fee)
    results.append({
        "max-blobs flag": "max-blobs="+str(i),
        "losses from using max-blob flag": f"{(total_prio_fee-filtered_prio_fee)/ll/1e9:.8f} ETH per proposed block",
        "...in USD (1 ETH = 3000 USD)": f"{(total_prio_fee-filtered_prio_fee)/ll/1e9*3000:.8f} USD",
        "% of total prio fee": f"{(total_prio_fee-filtered_prio_fee)/ll/1e9/medianprio*100:.4f} %"
    })

# Create a DataFrame from the results
df_results = pd.DataFrame(results)

# Print the DataFrame as a Markdown table (requires tabulate package)
print(df_results.to_markdown(index=False))


In [ ]:
gu2 = pd.read_parquet("gu2_data.parquet")
gu2 = pd.merge(base_fee, gu2, left_on="block_number", right_on="block_number")
gu2["block_basefee"] = gu2["block_gas_used"] * gu2["base_fee"] / 1e9
gu2["block_prio_fee"] = (gu2["block_cost"] - gu2["block_basefee"]) / gu2["block_gas_used"]
gu2["block_basefee"] = (gu2["block_basefee"]) / gu2["block_gas_used"]
gu2["date"] = gu2.slot.apply(lambda x: xatu.helpers.slot_to_day(x))
gu3 = gu2.groupby("date")[["block_prio_fee", "block_basefee"]].median().reset_index()
gu2 = gu2.groupby("date")[["block_prio_fee", "block_basefee"]].mean().reset_index()
gu2

medianprio = gu2.block_prio_fee.median() / 1e9
float(medianprio)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df2["date"],
        y=df2["prio_fee_per_gas"],
        mode="lines",
        name="Blob Transactions (mean)",
        line=dict(color="firebrick", width=2),
    )
)
fig.add_trace(
    go.Scatter(
        x=df3["date"],
        y=df3["prio_fee_per_gas"],
        mode="lines",
        name="Blob Transactions (median)",
        line=dict(color="firebrick", width=2,  dash="dash"),
    )
)

fig.add_trace(
    go.Scatter(
        x=gu2["date"],
        y=gu2["block_prio_fee"],
        mode="lines",
        name="Type 2 (mean)",
        line=dict(color="black", width=2),
       
    )
)

fig.add_trace(
    go.Scatter(
        x=gu3["date"],
        y=gu3["block_prio_fee"],
        mode="lines",
        name="Type 2 (median)",
        line=dict(color="black", width=2, dash="dash"),
       
    )
)

# Update layout for a clean, professional look
fig.update_layout(
    
    title='Prio Fee Type 2 & 3 Transactions <span style="font-size: 15;">({} - {})</span>'.format(xatu.helpers.slot_to_day(blobgas_gr.slot.min()), xatu.helpers.slot_to_day(blobgas_gr.slot.max())),

    xaxis_title=None,
    yaxis_title="Priority Fee (gwei/gas)",
    font=dict(
        family="Ubuntu mono",
        size=12,
        color="black"
    ),
    plot_bgcolor="white",
    paper_bgcolor="white",
    xaxis=dict(
        showgrid=True,
        gridcolor="lightgray",
        zeroline=True,
        zerolinewidth=2, zerolinecolor="black",
        tickangle=45
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor="lightgray",
        zeroline=True,
        zerolinewidth=2, zerolinecolor="black"
    ),
    legend=dict(
        #orientation="h",
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01,
        bgcolor="rgba(255, 255, 255, 0.5)",
        bordercolor="lightgray",
        borderwidth=1
    ),
    margin=dict(l=60, r=40, t=80, b=80)
)

fig.show()
